### This notebook does the following:
- processes raw ridership data
- performs feature selection and cleaning
- performs hourly aggregation
- saves results in one file

In [1]:
import pandas as pd
import glob
import os
from datetime import date, timedelta
import itertools

In [2]:
dataDir = '/home/urwa/Documents/NYU/Data/'
processedFile = '/home/urwa/Documents/NYU/JfkVehiceByHour.csv'

In [3]:
files = glob.glob(dataDir+'*csv')

In [4]:
zones = pd.read_csv('../Data/taxi_zones.csv')
zones.head(2)

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens


In [5]:
zones[zones.zone.apply(lambda x: 'Airport' in x)][['zone','LocationID']]

,zone,LocationID
0,Newark Airport,1
136,JFK Airport,132
145,LaGuardia Airport,138


In [6]:
JfK_zone = 132
validDestZones = list(set([z for z in zones.LocationID if z != JfK_zone]))
len(validDestZones)

259

In [7]:
for file in files:
    print("Processing "+str(file).split('/')[-1])
    
    vehicleType = str(file).split('/')[-1].split('_')[0]
    df = pd.read_csv(file)
    print("DataFrame Shape: "+str(df.shape))
    
    # rename columns for consistency
    # set passenger count to 1 for fhv
    if vehicleType == 'fhv':
        df.rename(columns={'Pickup_DateTime': 'tpep_pickup_datetime', \
                           'PUlocationID':'PULocationID', 'DOlocationID':'DOLocationID' },inplace=True)
        df['passenger_count'] = 1
        
    if vehicleType == 'green':
        df.rename(columns={'lpep_pickup_datetime': 'tpep_pickup_datetime'},inplace=True)


    # treat for na values
    df = df.dropna(subset=['tpep_pickup_datetime','PULocationID', 'DOLocationID'])
    df.fillna(value={'passenger_count':1}, inplace = True)
    
    # correct data types
    df['PULocationID'] = df['PULocationID'].astype('int')
    df['DOLocationID'] = df['DOLocationID'].astype('int')
    
    # filter to get outgoing traffic from JFK
    df = df[(df['PULocationID'] == JfK_zone) & (df['DOLocationID'].apply(lambda x: x in validDestZones))]
    print("JFK out DataFrame Shape: "+str(df.shape))
    
    # treat datetime
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['Date'] = df['tpep_pickup_datetime'].dt.date
    df['Hour'] = df['tpep_pickup_datetime'].dt.hour
    
    #df['vehicle_type'] = vehicleType
    
    # select rquired columns
    #df = df[['vehicle_type', 'Date', 'Hour', 'DOLocationID','passenger_count']]
    df = df[['Date', 'Hour', 'DOLocationID','passenger_count']]
    
    # Hourly aggregation
    #df_count = df.groupby(['vehicle_type', 'Date', 'Hour', 'DOLocationID']).count().reset_index()
    df_count = df.groupby(['Date', 'Hour', 'DOLocationID']).count().reset_index()
    df_count.rename(columns={'passenger_count': 'vehicle_count'},inplace=True)

    #df_sum = df.groupby(['vehicle_type', 'Date', 'Hour', 'DOLocationID']).sum().reset_index()

    #aggregatedDf = pd.merge(df_count,df_sum, on=['vehicle_type', 'Date', 'Hour', 'DOLocationID'], how='inner')
    aggregatedDf = df_count
    
    print("Aggregated DataFrame Shape: "+str(aggregatedDf.shape))
    print(aggregatedDf.head(3))
    # save file
    if os.path.exists(processedFile):
        print('append to results...')
        aggregatedDf.to_csv(processedFile,index=False, header=False, mode='a+')      
    else:
        print('create results file...')
        aggregatedDf.to_csv(processedFile,index=False)
    print('file saved..')
    print("------------------------------------------------")

Processing green_tripdata_2018-07.csv
DataFrame Shape: (684455, 19)
JFK out DataFrame Shape: (35, 19)
Aggregated DataFrame Shape: (35, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2018-07-01     1           121              1
1  2018-07-02    21           230              1
2  2018-07-03     6            86              1
create results file...
file saved..
------------------------------------------------
Processing yellow_tripdata_2018-02.csv
DataFrame Shape: (8492076, 17)
JFK out DataFrame Shape: (162101, 17)
Aggregated DataFrame Shape: (63667, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2008-12-31    14           137              1
1  2018-01-16    19            10              1
2  2018-01-16    22            28              1
append to results...
file saved..
------------------------------------------------
Processing fhv_tripdata_2017-10.csv
DataFrame Shape: (17890689, 6)
JFK out DataFrame Shape: (182087, 7)
Aggregated DataFrame Shape: (74067, 4)
         Dat

DataFrame Shape: (800084, 19)
JFK out DataFrame Shape: (51, 19)
Aggregated DataFrame Shape: (51, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2018-04-01     5            49              1
1  2018-04-01     7           181              1
2  2018-04-01    10           186              1
append to results...
file saved..
------------------------------------------------
Processing fhv_tripdata_2017-01.csv


/home/urwa/.virtualenvs/stan/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


DataFrame Shape: (13657212, 5)
JFK out DataFrame Shape: (0, 6)
Aggregated DataFrame Shape: (0, 4)
Empty DataFrame
Columns: [Date, Hour, DOLocationID, vehicle_count]
Index: []
append to results...
file saved..
------------------------------------------------
Processing yellow_tripdata_2018-10.csv
DataFrame Shape: (8821105, 17)
JFK out DataFrame Shape: (212255, 17)
Aggregated DataFrame Shape: (74624, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2008-12-31    23            50              1
1  2008-12-31    23           162              1
2  2009-01-01     5           186              1
append to results...
file saved..
------------------------------------------------
Processing fhv_tripdata_2017-05.csv
DataFrame Shape: (15397388, 5)
JFK out DataFrame Shape: (1118, 6)
Aggregated DataFrame Shape: (1017, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2017-05-01     5            13              1
1  2017-05-01     5           161              1
2  2017-05-01     7          

file saved..
------------------------------------------------
Processing fhv_tripdata_2018-12.csv
DataFrame Shape: (23854144, 7)
JFK out DataFrame Shape: (234343, 8)
Aggregated DataFrame Shape: (86376, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2018-12-01     0             4              2
1  2018-12-01     0             7              5
2  2018-12-01     0            10              3
append to results...
file saved..
------------------------------------------------
Processing fhv_tripdata_2018-03.csv
DataFrame Shape: (21985249, 7)
JFK out DataFrame Shape: (219062, 8)
Aggregated DataFrame Shape: (82490, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2018-03-01     0             3              1
1  2018-03-01     0             4              2
2  2018-03-01     0             7              3
append to results...
file saved..
------------------------------------------------
Processing green_tripdata_2017-09.csv
DataFrame Shape: (882464, 19)
JFK out DataFrame Shape: (

file saved..
------------------------------------------------
Processing yellow_tripdata_2018-04.csv
DataFrame Shape: (9305515, 17)
JFK out DataFrame Shape: (210272, 17)
Aggregated DataFrame Shape: (75371, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2009-01-01     0            72              1
1  2009-01-01    18           162              1
2  2009-01-01    20            39              1
append to results...
file saved..
------------------------------------------------
Processing yellow_tripdata_2018-03.csv
DataFrame Shape: (9430376, 17)
JFK out DataFrame Shape: (199739, 17)
Aggregated DataFrame Shape: (73962, 4)
         Date  Hour  DOLocationID  vehicle_count
0  2002-12-31    23           239              1
1  2003-01-01    23            87              1
2  2009-01-01     0            89              1
append to results...
file saved..
------------------------------------------------
Processing fhv_tripdata_2018-02.csv
DataFrame Shape: (19350693, 7)
JFK out DataFrame S

### Further processing

In [8]:
def getcCompleteGridDf(minDate,maxDate, locations):
    minDate = [int(x) for x in minDate.split('-')]
    maxDate = [int(x) for x in maxDate.split('-')]
    sdate = date(minDate[0], minDate[1], minDate[2])   
    edate = date(maxDate[0], maxDate[1], maxDate[2])    

    delta = edate - sdate       
    days = []
    for i in range(delta.days + 1):
        days.append(sdate + timedelta(days=i))
    hours = list(range(24))
    print(len(days))
    print(len(hours))
    
    combList = list(itertools.product(*[days,hours,locations]))
    dfList = [{'Date':d, 'Hour':h, 'DOLocationID':l} for d,h,l in combList]
 
    dateHourDf = pd.DataFrame(dfList)
    dateHourDf['Date'] = pd.to_datetime(dateHourDf['Date']).dt.date
    return dateHourDf

In [9]:
processedDf = pd.read_csv(processedFile)
processedDf.head(2)

,Date,Hour,DOLocationID,vehicle_count
0,2018-07-01,1,121,1
1,2018-07-02,21,230,1


In [10]:
processedDf.shape

(3341756, 4)

In [11]:
# ensuring proper grouping since files were grouped by independently
processedDf = processedDf.groupby(['Date', 'Hour', 'DOLocationID']).sum().reset_index()
processedDf.shape

(2332343, 4)

In [12]:
# sanity checks
validYears = [2017,2018]
processedDf = processedDf[processedDf.Date.apply(lambda x: int(x.split('-')[0]) in validYears)]

validMonths = list(range(1,13))
processedDf = processedDf[processedDf.Date.apply(lambda x: int(x.split('-')[1]) in validMonths)]

processedDf.shape    

(2332298, 4)

In [13]:
minDate, maxDate = (processedDf.Date.min(), processedDf.Date.max()) 
#v_types = list(set(processedDf.vehicle_type))
locations = list(set(processedDf.DOLocationID))

#print(len(v_types))
print(len(locations))

dateHourDf = getcCompleteGridDf(minDate,maxDate,locations)
dateHourDf.shape

258
730
24


(4520160, 3)

In [22]:
dateHourDf['Date'] = pd.to_datetime(dateHourDf['Date'])
processedDf['Date'] = pd.to_datetime(processedDf['Date'])

In [28]:
mergedDf = pd.merge(dateHourDf,processedDf, on=['Date', 'Hour', 'DOLocationID'], how='left')
mergedDf.fillna(0, inplace=True)
mergedDf['Date'] = mergedDf['Date'].dt.date
print(mergedDf.shape)
mergedDf.head(3)

(4520160, 4)


,Date,Hour,DOLocationID,vehicle_count
0,2017-01-01,0,1,0.0
1,2017-01-01,0,2,0.0
2,2017-01-01,0,3,0.0


In [29]:
# sanity check
print(processedDf.vehicle_count.sum())
print(mergedDf.vehicle_count.sum())

8928716
8928716.0


In [30]:
# sanity check for size of new dataframe
730*258*24

4520160

In [31]:
# fraction of combinations that have data. Data is very scarce !!!
2332298/4520160

0.5159768680754664

In [32]:
mergedDf.to_csv(processedFile,index=False)